## import libraries

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## load dataset

In [38]:
raw = pd.read_csv('steam_games.csv')
raw.sample(5)

,title,url,image,release_date,platforms,discount_rate,original_price,discounted_price,developer,publisher,overall_reviews,text_reviews,description,tags,processor,ram,graphic_card,rating,language,metacriticts
21265,東周列萌志 Philosophic Love－原聲帶組合包 OST Bundle（升級包 U...,https://store.steampowered.com/bundle/13561/_P...,https://cdn.akamai.steamstatic.com/steam/bundl...,NaN,"Windows,Mac OS",NaN,Rp 142 198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26991,Treasure Adventure Game,https://store.steampowered.com/app/863940/Trea...,https://cdn.akamai.steamstatic.com/steam/apps/...,"19 Mar, 2021",Windows,NaN,Free,NaN,Robit Games,Robit Games,Positive,- 90% of the 20 user reviews for this game are...,Treasure Adventure Game casts you in the role ...,"Metroidvania,2D Platformer,Action,Puzzle Platf...",1.8 GHz Processor,512 MB RAM,3D graphics card compatible with DirectX 7,NaN,English,NaN
52022,Slime Research,https://store.steampowered.com/app/966180/Slim...,https://cdn.akamai.steamstatic.com/steam/apps/...,"2 Jan, 2019","Windows,Mac OS,Linux",NaN,Rp 39 999,NaN,Schism LLC,Schism LLC,3 user reviews,- Need more user reviews to generate a score,Slimy Puzzle Platforming,"Physics,2D Platformer,Controller,Level Editor,...",SSE2 instruction set support,200 MB RAM,Graphics card with DX10 (shader model 4.0) ca...,NaN,English,NaN
6965,Svoboda 1945: Liberation,https://store.steampowered.com/app/1076620/Svo...,https://cdn.akamai.steamstatic.com/steam/apps/...,"3 Aug, 2021","Windows,Mac OS,Linux",NaN,Rp 99 999,NaN,Charles Games,Charles Games,Very Positive,- 97% of the 71 user reviews for this game are...,Svoboda 1945: Liberation is a unique blend of ...,"Singleplayer,Visual Novel,Emotional,Adventure,...",1.6 GHz,8 GB RAM,HD Graphics 530 or better,NaN,"English,Czech,German",81.0
9337,Super Chibi Knight,https://store.steampowered.com/app/283080/Supe...,https://cdn.akamai.steamstatic.com/steam/apps/...,"24 Jun, 2015","Windows,Mac OS,Linux",NaN,Rp 45 999,NaN,PestoForce,"Armor Games Studios,PestoForce",Very Positive,- 88% of the 278 user reviews for this game ar...,The Kingdom of Oukoku is threatened by the spi...,"Action,Adventure,Indie,RPG,2D,Action RPG,Singl...",2.5 GHz,1 GB RAM,Direct X9.0c Compatible Card,NaN,"English,French,Italian,German,Spanish - Spain,...",NaN


## data understanding

In [39]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68468 entries, 0 to 68467
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             68468 non-null  object 
 1   url               68468 non-null  object 
 2   image             68468 non-null  object 
 3   release_date      62160 non-null  object 
 4   platforms         66194 non-null  object 
 5   discount_rate     5401 non-null   object 
 6   original_price    67795 non-null  object 
 7   discounted_price  5401 non-null   object 
 8   developer         61855 non-null  object 
 9   publisher         56815 non-null  object 
 10  overall_reviews   55479 non-null  object 
 11  text_reviews      55479 non-null  object 
 12  description       40956 non-null  object 
 13  tags              61039 non-null  object 
 14  processor         55299 non-null  object 
 15  ram               55091 non-null  object 
 16  graphic_card      51361 non-null  object

Observations:
- `url`,`image`,`discount_rate`,`discounted_price`,`description` drop
- `release_date` extract month and year
- `platfoms`,`developer`,`publisher`,`tags` split
- `original_price` change data type (extract price)
- `text_reviews` extract rating (`user_rating`) and total user reviews (`total_reviews`)
- `language` extract number of language (`language`)
- `metacriticts` too much missing values, drop

In [40]:
raw.describe()

,metacriticts
count,1881.000000
mean,80.971823
std,4.458169
min,75.000000
25%,77.000000
50%,80.000000
75%,84.000000
max,97.000000


In [41]:
raw.describe(include='object')

,title,url,image,release_date,platforms,discount_rate,original_price,discounted_price,developer,publisher,overall_reviews,text_reviews,description,tags,processor,ram,graphic_card,rating,language
count,68468,68468,68468,62160,66194,5401,67795,5401,61855,56815,55479,55479,40956,61039,55299,55091,51361,13428,59912
unique,64887,68468,65033,3894,12,89,4591,1500,24765,20134,18,13440,39011,40137,12423,859,14567,1,7879
top,Game + Soundtrack,https://store.steampowered.com/app/730/Counter...,https://cdn.akamai.steamstatic.com/steam/apps/...,"14 Oct, 2020",Windows,-50%,Rp 69 999,Rp 4 249,Ubisoft - San Francisco,Degica,Very Positive,- Need more user reviews to generate a score,Minimal physical puzzle with explosions,Action,2.66 GHz Intel Core2 Duo E6750 or 2.8 GHz AMD ...,4 GB RAM,256 MB DirectX 9 / NVIDIA® GeForce® 8600 GT or...,Rating for: PEGI,English
freq,13,1,6,146,44961,1006,5108,329,1167,1049,11078,15169,15,1715,801,13411,1167,13428,24115


observations:
- `rating` data is wrong, consider drop or rescrape data
- `tags` too much unique value, consider to create new columns tag that has been aggregated

## data preprocessing (format)

### drop duplicates and uninformative columns

In [42]:
df_drop = raw.copy()
df_drop.drop_duplicates(subset='title',inplace=True)
df_drop['title'].duplicated().sum()

0

In [43]:
drop = ['url','image','discount_rate','discounted_price','description','metacriticts','processor','graphic_card'] #these variables unable to show any kind of valuable information that can be used for current analysis and machine learning model, consider to be used for other analysis
df_drop = df_drop.drop(labels=drop,axis=1)
df_drop.sample(10)

,title,release_date,platforms,original_price,developer,publisher,overall_reviews,text_reviews,tags,ram,rating,language
12737,Injustice™ 2 - Enchantress,"16 Jan, 2018",Windows,Rp 48 999,"NetherRealm Studios,QLOC",NaN,Positive,- 84% of the 13 user reviews for this game are...,Action,4 GB RAM,Rating for: PEGI,"English,French,Italian,German,Spanish - Spain,..."
31964,EXCHANGE,"12 Oct, 2018",Windows,Rp 39 999,Self Destruction Games,Self Destruction Games,Mixed,- 66% of the 18 user reviews for this game are...,"Indie,Action,Massively Multiplayer,Casual,Earl...",2 GB RAM,NaN,"English,Turkish"
58937,Rogue Port - Blue Nightmare,"31 Mar, 2017",Windows,Rp 8 500,Volens Nolens Games,Volens Nolens Games,Mixed,- 64% of the 14 user reviews for this game are...,"Action,Adventure,RPG,Indie,Pixel Graphics,RPGM...",NaN,NaN,English
63034,Train Simulator: DB BR 361 Loco Add-On,"18 Dec, 2015",Windows,Rp 135 999,Dovetail Games,Dovetail Games - Trains,Positive,- 93% of the 15 user reviews for this game are...,Simulation,2 GB RAM,Rating for: PEGI,"English,French,German"
2662,PAYDAY 2: Gunslinger Weapon Pack,"11 Nov, 2020","Windows,Linux",Rp 25 999,OVERKILL - a Starbreeze Studio.,NaN,Mostly Positive,- 78% of the 123 user reviews for this game ar...,"RPG,Action,Gore,Violent",4 GB RAM,Rating for: PEGI,"English,German,French,Italian,Spanish - Spain,..."
20887,Farm and Train Bundle,NaN,Windows,Rp 215 998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61860,CHUCHEL Cherry Edition,NaN,"Windows,Mac OS",Rp 94 998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5925,Rags to Dishes,"11 Nov, 2020",Windows,Rp 108 999,Novity,Novity,Very Positive,- 97% of the 102 user reviews for this game ar...,"Simulation,Casual,VR,Singleplayer,First-Person...",8 GB RAM,Rating for: PEGI,English
31486,Artizens,"5 Mar, 2015",Windows,NaN,"Artizens, Inc.","Artizens, Inc.",Mixed,- 48% of the 31 user reviews for this game are...,"Early Access,Action,RPG,Indie",2 GB RAM,NaN,English
27863,Simple Man,"12 Mar, 2018",Windows,Rp 17 499,CSM,W.T.B.,Mixed,- 47% of the 17 user reviews for this game are...,"Action,Adventure,RPG,Indie",512 MB RAM,NaN,"English,Russian"


### data extraction and formatting

#### extract `month` and `year`

In [44]:
#define function to extract year
def extract_year(x):
    year = None
    if type(x) == list:
        for i in x:
            if len(i) == 4:
                year = i
                return year
            else: 
                year = None
        return year
    else:
        year = x
    return year

In [45]:
#df_drop[['date','month','year']] = df_drop['release_date'].str.split(' ', n=2 , expand=True)
df_extract = df_drop.copy()
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#df_extract['year'] = df_extract['release_date'].str[-4:].apply(pd.to_numeric, errors=('coerce')) 
df_extract['year'] = df_extract['release_date'].str.findall('(\d+)').apply(lambda x: extract_year(x)).astype(float)
df_extract['month'] = df_extract['release_date'].str.findall("[a-zA-Z]+").str[0]
df_extract['month'] = df_extract['month'].replace(['August','June'],['Aug','Jun'])
df_extract.loc[~df_extract['month'].isin(month),'month'] = None
df_extract = df_extract.drop('release_date', axis=1) #drop release date
df_extract.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,text_reviews,tags,ram,rating,language,year,month
43611,GunHero,"Windows,Linux",Rp 59 999,Olli-Samuli Lehmus,Olli-Samuli Lehmus,4 user reviews,- Need more user reviews to generate a score,"Action,Indie,Platformer",1 GB RAM,NaN,"English,Finnish",2017.0,Apr
61766,Scoregasm,"Windows,Mac OS",Rp 45 999,RC Knight,Charlie's Games,Very Positive,- 83% of the 79 user reviews for this game are...,"Action,Indie,Twin Stick Shooter,Bullet Hell,Sc...",512mb\n\t\t\t\t\t\t\t\t\t\t\t\t\t,NaN,English,2012.0,Feb
32815,Total War: SHOGUN 2 - Dragon War Battle Pack,"Windows,Mac OS,Linux",Rp 53 999,"The Creative Assembly,Feral Interactive (Mac),...","SEGA,Feral Interactive (Mac),Feral Interactive...",Mixed,- 62% of the 101 user reviews for this game ar...,Strategy,"1GB RAM (XP), 2GB RAM (Vista / Windows7)",Rating for: PEGI,"English,Czech,French,German,Italian,Polish,Rus...",2012.0,May
28993,Magic gems,Windows,Rp 83 999,Feanus,kovalevviktor,Mixed,- 69% of the 13 user reviews for this game are...,"Simulation,2D,Runner,Colorful,Pixel Graphics,F...",1024 MB RAM,NaN,English,2021.0,Jul
22197,The Witcher 3: Wild Hunt - Nilfgaardian Armor Set,Windows,Free,CD PROJEKT RED,NaN,Very Positive,- 92% of the 328 user reviews for this game ar...,"Free to Play,RPG",6 GB RAM,Rating for: PEGI,"English,French,Italian,German,Spanish - Spain,...",2015.0,Jun


#### extract `user_rating` and `total_reviews`

In [46]:
df_extract_2 = df_extract.copy()
df_extract_2['text_reviews'] = df_extract_2['text_reviews'].str.replace(',','')
df_extract_2['user_rating'] = df_extract_2['text_reviews'].str.findall('(\d+)').str[0].astype(float)
df_extract_2['total_review'] = df_extract_2['text_reviews'].str.findall('(\d+)').str[1].astype(float)
df_extract_2 = df_extract_2.drop('text_reviews',axis=1)
df_extract_2.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,tags,ram,rating,language,year,month,user_rating,total_review
42134,V-Skin Webcam Hand Expansion Pack,"Windows,Mac OS",Rp 424 999,"Q.B.Q.F MULTIMEDIA CO.,LTD.","Q.B.Q.F MULTIMEDIA CO.,LTD.",NaN,"Video Production,Audio Production,Web Publishi...",4 GB RAM,NaN,"English,Japanese,Simplified Chinese,Traditiona...",2022.0,Feb,NaN,NaN
63195,Neko Hacker Plus,Windows,Rp 24 999,Axyos Games,Axyos Games,Positive,"Nudity,NSFW,Cute,Puzzle,Anime,Casual,Sexual Co...",4 GB RAM,NaN,English,2021.0,Sep,80.0,26.0
54756,Midnight's Blessing 2,Windows,Rp 69 999,Warfare Studios,Warfare Studios,6 user reviews,"Adventure,Indie,Casual,Simulation,RPG,Strategy...",128 MB RAM,NaN,English,2017.0,Jan,NaN,NaN
51727,Tennis World Tour - Caroline Garcia,Windows,Rp 24 999,Breakpoint,Nacon,NaN,Sports,NaN,Rating for: PEGI,"English,French,Italian,German,Spanish - Spain,...",2020.0,Apr,NaN,NaN
41162,Watch_Dogs® 2 - EliteSec Pack,Windows,Rp 70 000,Ubisoft,NaN,8 user reviews,"Action,Adventure",6 GB RAM,NaN,"English,French,Italian,German,Spanish - Spain,...",2016.0,Dec,NaN,NaN


#### extract `supported_language`

In [47]:
df_extract_3 = df_extract_2.copy()
df_extract_3['supported_language'] = df_extract_3['language'].str.split(',').apply(lambda x: len(x) if type(x) == list else x)
#df_extract_3['english_language'] = df_extract_3['language'].str.lower().str.contains('english')
df_extract_3 = df_extract_3.drop('language',axis=1)
df_extract_3.sample(5)

,title,platforms,original_price,developer,publisher,overall_reviews,tags,ram,rating,year,month,user_rating,total_review,supported_language
35159,Nobunaga's Ambition: Taishi - シナリオ「長篠の戦い」/Scen...,Windows,Rp 37 000,"KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.",3 user reviews,Simulation,2 GB RAM,Rating for: PEGI,2018.0,Jun,NaN,NaN,3.0
52778,The Legend of Heroes: Trails of Cold Steel III...,Windows,Rp 8 499,"Nihon Falcom,Engine Software BV,PH3 GmbH","NIS America, Inc.",1 user reviews,RPG,NaN,NaN,2020.0,Mar,NaN,NaN,3.0
57565,Die With Glory — Point and Click Adventure Game,"Windows,Mac OS",Rp 48 999,Castle,Veslo Games,Mixed,"Adventure,Action,Indie,Point & Click,Funny,2D,...",512 MB RAM,NaN,2017.0,May,46.0,32.0,2.0
39350,TS Marketplace: Amtrak E8 Scenario Pack 01 Add-On,Windows,Rp 69 999,High Iron Simulations,Dovetail Games - Trains,1 user reviews,Simulation,4 GB RAM,Rating for: PEGI,2018.0,Jun,NaN,NaN,1.0
8728,ONE PIECE: PIRATE WARRIORS 4 Whole Cake Island...,Windows,Rp 119 000,Koei Tecmo,BANDAI NAMCO Entertainment,Positive,Action,8 GB RAM,Rating for: PEGI,2020.0,Jul,83.0,30.0,13.0


#### extract `price`

In [48]:
df_price = df_extract_3.copy()
df_price['price'] = df_price['original_price'].str.lower().str.strip('rp ').str.replace(' ','').astype(int,errors='ignore')
df_price.loc[df_price['price'].str.contains('free|demo|season|third|now',na=False),'price'] = 0
df_price['price'] = df_price['price'].astype(float)
df_price = df_price.drop('original_price',axis=1)
df_price.sample(5)

,title,platforms,developer,publisher,overall_reviews,tags,ram,rating,year,month,user_rating,total_review,supported_language,price
4029,Owlboy,"Windows,Mac OS,Linux",D-Pad Studio,D-Pad Studio,Mostly Positive,"Pixel Graphics,Adventure,Indie,Great Soundtrac...",1 GB RAM,Rating for: PEGI,2016.0,Nov,75.0,12.0,13.0,169999.0
8299,Secret Agent HD,"Windows,Linux",Emberheart Games,Apogee Entertainment,Very Positive,"Action,Adventure,Puzzle Platformer,Platformer,...",512 MB RAM,NaN,2021.0,Jun,98.0,79.0,1.0,59999.0
56448,Animalia New African Skins,Windows,High Brazil Studio,High Brazil Studio,4 user reviews,"Action,Adventure,Indie,Massively Multiplayer,S...",NaN,NaN,2022.0,Feb,NaN,NaN,1.0,69999.0
60866,Output Pasture,Windows,DuskDogStudio,"DuskDogStudio,HeartBeat Games",Positive,"Strategy,Indie,Casual,Tower Defense,2D,Roguelike",4 GB RAM,NaN,2020.0,Mar,88.0,45.0,5.0,64999.0
20069,Word Wonders: The Tower of Babel,Windows,Chimera Entertainment,GAMES in FLAMES,Positive,"Casual,Word Game",1 GB RAM,NaN,2015.0,May,100.0,15.0,1.0,32999.0


#### extract `rated_by_pegi`

In [49]:
df_pegi = df_price.copy()
df_pegi['pegi_rated'] = np.where(df_pegi['rating']=='Rating for: PEGI', 1, 0)
df_pegi = df_pegi.drop('rating',axis=1)
df_pegi.sample(5)

,title,platforms,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated
9400,"""ONEECHANBARA WORLD"" BUNDLE",Windows,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,1338998.0,0
57006,Gauntlet of IRE,Windows,KillerPokeGames,KillerPokeGames,Mostly Positive,"Casual,Action,Indie,Difficult,Isometric,Side S...",4 GB RAM,2017.0,Dec,76.0,158.0,1.0,0.0,1
68424,GUILTY GEAR -STRIVE-,Windows,Arc System Works,Arc System Works,Very Positive,"2D Fighter,Fighting,Anime,2.5D,PvP,Arcade,Grea...",4 GB RAM,2021.0,Jun,93.0,527.0,9.0,749000.0,0
5375,Assassin’s Creed Unity Revolutionary Armaments...,Windows,Ubisoft,Ubisoft,Mostly Negative,"Action,Adventure",NaN,2014.0,Nov,30.0,13.0,14.0,30000.0,1
12684,Train Simulator: Union Pacific U50 Loco Add-On,Windows,Digital Train Model,Dovetail Games - Trains,Mixed,Simulation,4 GB RAM,2018.0,May,63.0,11.0,1.0,108999.0,1


#### extract `windows`, `mac`, and `linux`

In [50]:
df_platform = df_pegi.copy()
df_platform['windows'] = np.where(df_platform['platforms'].str.contains('Windows',na=False), 1, 0)
df_platform['mac'] = np.where(df_platform['platforms'].str.contains('Mac OS',na=False), 1, 0)
df_platform['linux'] = np.where(df_platform['platforms'].str.contains('Linux',na=False), 1, 0)
df_platform['VR'] = np.where(df_platform['platforms'].str.contains('VR Supported',na=False), 1, 0)
df_platform = df_platform.drop('platforms',axis=1)
df_platform.sample(10)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
61448,宥蘿的奇幻冒險 YURO'S FANTASY ADVENTURE,戴永翔(Dai Yong Xiang),戴永翔(Dai Yong Xiang),Positive,"RPG,Casual,Anime,JRPG,Cute,Adventure,Funny,Act...",8 GB RAM,2020.0,Nov,100.0,18.0,2.0,24999.0,1,1,0,0,0
55535,MELTY BLOOD: TYPE LUMINA - Round Announcements...,"FRENCH-BREAD,TYPE-MOON,DELiGHTWORKS Inc.,TAMSO...",DELiGHTWORKS Inc.,3 user reviews,"Action,PvP,Multiplayer,2D,Pixel Graphics,Vampi...",4 GB RAM,2021.0,Sep,NaN,NaN,5.0,249999.0,1,1,0,0,0
27392,Runaway Express Mystery,Icarus Games,Libredia,Mixed,"Casual,Hidden Object,Point & Click",1024 MB RAM,2014.0,Sep,68.0,22.0,7.0,59999.0,0,1,0,0,0
10784,Fluffy Store - Listening with you,AsicxArt,SakuraGame,Positive,"Adventure,RPG,Indie,Casual",1 GB RAM,2019.0,Sep,100.0,30.0,4.0,17499.0,0,1,1,0,0
24514,Armored Train,NORSKA,NORSKA,Mixed,"Strategy,Adventure,Indie,RPG,Simulation,Pixel ...",512 MB RAM,2018.0,Sep,47.0,17.0,2.0,8499.0,0,1,0,0,0
38983,Heroes & Generals - GE Soldier Pack: Medium Ta...,TLM Partners,TLM Partners,4 user reviews,"Strategy,Action,Free to Play,Massively Multipl...",4 GB RAM,2020.0,May,NaN,NaN,11.0,169999.0,0,1,0,0,0
9523,Tangledeep Complete (Includes Both Expansions ...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,166826.0,0,1,1,1,0
44695,JS Page,Geargom,Geargom,NaN,"Utilities,Education,Web Publishing,Software Tr...",4 GB RAM,2021.0,Mar,NaN,NaN,6.0,139999.0,0,1,0,0,0
6677,Borderlands 2: Gunzerker Madness Pack,"Gearbox Software,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)",Mostly Positive,"Action,RPG",2 GB(XP)/ 2 GB(Vista),2013.0,Jan,73.0,76.0,7.0,9299.0,1,1,1,1,0
53416,Colorless Life,HIPIX,HIPIX,9 user reviews,"Casual,Adventure,Indie,Action,Simulation,Horro...",NaN,2017.0,Jul,NaN,NaN,2.0,18999.0,0,1,0,0,0


#### remap `ram`

In [68]:
df_ram = df_platform.copy()
df_ram.loc[df_ram['ram'].str.lower().str.contains('9 gb', na=False),'ram'] = '9 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('8 gb|8000 mb|８ gb|8gb|8.0 gb|8\+ gb', na=False),'ram'] = '8 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('7 gb', na=False),'ram'] = '7 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('6 gb|6000 mb|６ gb', na=False),'ram'] = '6 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('5 gb|5000 mb', na=False),'ram'] = '5 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('4 gb|4096 mb|4000 mb|4gb|４ gb|4.00 gb|4g|4 ram gb', na=False),'ram'] = '4 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('3 gb|3000 mb|3 go mb|3gb', na=False),'ram'] = '3 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('2 gb|2048|2000 mb|2gb|1536 mb|2.0 gb|2000 gb|２ gb|1.5gb|2.0gb|2g', na=False),'ram'] = '2 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('1 gb|1024|1000 mb|1gb|800 mb|750 mb|1280 mb|700 mb|900 mb|１ gb|1\+', na=False),'ram'] = '1 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('512|500 mb|400 mb|512mb|600 mb|597 mb|320 mb|420 mb|500mb|349 mb', na=False),'ram'] = '512 MB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('256 gb|200 mb|300 mb|250 mb|256mb|220 mb|256 ram', na=False),'ram'] = '256 MB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('16.0 gb', na=False),'ram'] = '16 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('12.00 gb', na=False),'ram'] = '12 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('10 gb', na=False),'ram'] = '10 GB'
df_ram.loc[df_ram['ram'].str.lower().str.contains('128 mb|64m|70 mb|100 mb|150 mb|16 mb|8 mb|2 mb|1 mb|4 mb|50 mb|3 mb|6 mb|10 mb|128mb|80 mb|4 ram mb|30 mb|40 mb|25 mb|5 mb|120 mb|64 mb|90 mb|60 mb|95 mb|65 mb|64mb|16mb|100以上|32mb|97 mb|20 mb|99 mb|4mb|69 mb|59 mb', na=False),'ram'] = '<128 MB'
df_ram.loc[~df_ram['ram'].str.lower().str.contains('16 gb|12 gb|10 gb|9 gb|8 gb|7 gb|6 gb|5 gb|4 gb|3 gb|2 gb|1 gb|512 mb|256 mb|<128 mb', na=False),'ram'] = 'Unknown'
df_ram['ram'].value_counts()

4 GB       13536
2 GB       13186
Unknown    11849
<128 MB     8638
1 GB        8510
8 GB        6431
6 GB        1945
3 GB         658
5 GB         109
10 GB         17
7 GB           4
9 GB           2
12 GB          1
16 GB          1
Name: ram, dtype: int64

In [71]:
df_ram.sample(10)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
57198,Life of Lon: Chapter 1,Block Interval,Block Interval,Mostly Positive,"Adventure,VR,Puzzle",4 GB,2017.0,Jun,72.0,43.0,1.0,0.0,0,1,0,0,0
22320,The Witcher 3: Wild Hunt - NEW GAME +,CD PROJEKT RED,NaN,Very Positive,RPG,6 GB,2015.0,Aug,91.0,463.0,16.0,0.0,1,1,0,0,0
50679,Mondrian Squares,LH Games Limited,LH Games Limited,3 user reviews,"Strategy,Casual,Puzzle,Procedural Generation,L...",Unknown,2021.0,Dec,NaN,NaN,1.0,8499.0,0,1,0,0,0
8797,Tricky Towers - Candy Bricks,WeirdBeard,NaN,Positive,"Indie,Casual",1 GB,2017.0,Feb,100.0,20.0,18.0,9299.0,1,1,1,1,0
57866,Riders 2491,GoeX Games,GoeX Games,Mixed,"Action,Arcade,Shooter,Bullet Hell,Shoot 'Em Up...",2 GB,2020.0,Jul,63.0,11.0,1.0,24999.0,0,1,1,1,0
18858,StarShip Constructor,SSCTEAM,SSCTEAM,Mixed,"Indie,Space,Building,Early Access,Sandbox,2D",2 GB,2017.0,Jul,67.0,237.0,2.0,45999.0,0,1,0,0,0
4172,OMSI 2 Add-on Rheinhausen,Halycon Media,Halycon Media,Very Positive,"Simulation,Driving,Realistic,Singleplayer,Modd...",4 GB,2016.0,Oct,80.0,115.0,2.0,253532.0,0,1,0,0,0
3058,Coloring Game 4,L. Stotch,L. Stotch,Very Positive,"Free to Play,Indie,Casual,Design & Illustratio...",1 GB,2021.0,Jul,86.0,23.0,29.0,0.0,0,1,0,0,0
16570,Space Kettle,Maxkres Games,Maxkres Games,Positive,"Difficult,Singleplayer,Physics,Psychological H...",4 GB,2021.0,Feb,82.0,28.0,1.0,49999.0,0,1,1,0,0
16705,Driver Fusion - The Best Driver & Device Solution,Treexy,Treexy,Mostly Positive,"Free to Play,Indie,Utilities,Software Training...",<128 MB,2013.0,May,72.0,321.0,29.0,NaN,0,1,0,0,0


#### remap `overall_reviews`

In [72]:
df_or = df_ram.copy()
df_or.loc[~df_or['overall_reviews'].str.contains('Positive|Negative|Very|Overwhelmingly|Mixed|Mostly', na=False),'overall_reviews'] = '<10 reviews'
df_or['overall_reviews'].value_counts()

<10 reviews                26699
Very Positive              10628
Mixed                       9458
Positive                    9079
Mostly Positive             6749
Mostly Negative             1559
Overwhelmingly Positive      417
Negative                     234
Very Negative                 57
Overwhelmingly Negative        7
Name: overall_reviews, dtype: int64

In [73]:
df_or.sample(5)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
6854,Araha : Curse of Yieun Island,Palmsoft,Palmsoft,Mixed,"Indie,Adventure,Horror,Survival Horror",8 GB,2020.0,Jan,62.0,197.0,10.0,99999.0,0,1,0,0,0
55589,School Idol,Fury Games Production,Fury Games Production,Mostly Negative,"Sexual Content,Nudity,Violent,Action,Adventure...",4 GB,2018.0,Jul,24.0,29.0,3.0,17499.0,0,1,0,0,0
29721,TD Worlds + Original Soundtrack Bundle,NaN,NaN,<10 reviews,NaN,Unknown,NaN,None,NaN,NaN,NaN,106398.0,0,1,0,0,0
14896,人间-OST,白露社,PeriScope Game,Positive,"Adventure,Casual,Simulation,Indie",2 GB,2019.0,May,100.0,25.0,2.0,8499.0,0,1,0,0,0
38692,White Day - Apple School Uniform - Hee-Min Lee,SONNORI Corp,"PQube,SONNORI Corp",<10 reviews,"Action,Adventure,Indie",4 GB,2017.0,Aug,NaN,NaN,9.0,18999.0,0,1,0,0,0


#### reformat and remap `tags`

In [74]:
df_tags = df_or.copy()
df_tags.loc[df_tags['tags'].isna()==True,'tags'] = 'No Tags'
df_tags['tags'] = df_tags['tags'].str.split(',')
#df_tags['tags'] = df_tags['tags'].replace(',', '","')
#df_tags['tags'] = '["' + df_tags['tags'] + '"]'
df_tags.sample(10)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
17661,Empire Live,Osmose Studio,Osmose Studio,Positive,"[Indie, Strategy, RPG, Early Access, Multiplayer]",4 GB,2020.0,May,84.0,13.0,2.0,82999.0,0,1,0,0,0
41254,FORCED SHOWDOWN - Deluxe Edition Content,BetaDwarf,NaN,<10 reviews,"[Strategy, Action, RPG, Indie]",4 GB,2016.0,Mar,NaN,NaN,1.0,59999.0,0,1,1,1,0
24555,Odd||Even,kylix studio,kylix studio,Mixed,"[Experimental, Difficult, Education, Puzzle, L...",<128 MB,2016.0,Apr,53.0,30.0,1.0,108999.0,0,1,0,0,0
31032,FROM WRECK TO WRECK,NaN,NaN,<10 reviews,[No Tags],Unknown,NaN,None,NaN,NaN,NaN,179098.0,0,1,0,0,0
64773,"Warhammer 40,000: Inquisitor - Martyr - Monota...",NeocoreGames,NeocoreGames,Mixed,"[Action, Adventure, RPG, Indie, Gore, Violent]",4 GB,2018.0,Nov,63.0,11.0,10.0,17499.0,0,1,0,0,0
64183,Lance A Lot: Classic Edition,Brimstone,Brimstone,Very Positive,"[Indie, Action, Casual, 4 Player Local, Local ...",2 GB,2016.0,Oct,85.0,55.0,11.0,39999.0,0,1,0,0,0
33375,Idle Champions - Star the Displacer Beast Kitt...,Codename Entertainment Inc.,Codename Entertainment Inc.,<10 reviews,"[Strategy, Action, Adventure, Free to Play, In...",Unknown,2021.0,Nov,NaN,NaN,1.0,69999.0,0,1,1,0,0
16902,Lanternium,Blazing Planet Studio,Blazing Planet Studio,Positive,"[Puzzle Platformer, Character Action Game, Dif...",2 GB,2019.0,Apr,86.0,36.0,11.0,69999.0,0,1,1,1,0
8420,Adorable Crush,Mature Games,Mature Games,Mostly Positive,"[Dating Sim, Hentai, Casual, Visual Novel, Mat...",1 GB,2020.0,Oct,77.0,515.0,4.0,24999.0,0,1,0,0,0
63677,Shadow Ninja: Apocalypse,Boogygames Studios,Boogygames Studios,Mixed,"[Action, Indie, Adventure, Casual, Strategy, P...",1 GB,2015.0,Nov,57.0,132.0,1.0,18999.0,1,1,1,0,0


In [75]:
def to_1D(series): #
 return pd.Series([x for _list in series for x in _list])

#### reformat and remap `developer`

In [76]:
df_dev = df_tags.copy()
df_dev.loc[df_dev['developer'].isna()==True,'developer'] = 'Unknown'
df_dev['developer'] = df_dev['developer'].str.split(',')
#df_tags['tags'] = df_tags['tags'].replace(',', '","')
#df_tags['tags'] = '["' + df_tags['tags'] + '"]'
df_dev.sample(10)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
13784,RPGツクールMV×パルフォン,[Unknown],NaN,<10 reviews,[No Tags],Unknown,NaN,None,NaN,NaN,NaN,518998.0,0,1,0,0,0
7099,KAPIA,[2 FOR 2],2 FOR 2,Positive,"[Comedy, Adventure, Point & Click, Puzzle, Fun...",4 GB,2022.0,Jan,100.0,19.0,7.0,108999.0,0,1,0,0,0
50218,Trigger Dungeon,[171Dev],171Dev,<10 reviews,"[Action, Platformer, 2D Platformer, Precision ...",<128 MB,2022.0,Jan,NaN,NaN,1.0,24999.0,0,1,0,0,0
33152,War Thunder - Chinese Starter Pack,[Gaijin Entertainment],Gaijin Distribution KFT,<10 reviews,"[Action, Free to Play, Massively Multiplayer, ...",4 GB,2022.0,Mar,NaN,NaN,18.0,139999.0,1,1,1,1,0
34384,Pixel Shooter,[Fabio Cunha],Fabio Cunha,Mostly Negative,"[Side Scroller, 2D Platformer, Casual, Difficu...",2 GB,2017.0,Jul,32.0,31.0,1.0,84999.0,0,1,0,0,0
937,Sally Face - Episode One,[Portable Moose],Portable Moose,Overwhelmingly Positive,"[Story Rich, Psychological Horror, Dark, Horro...",4 GB,2016.0,Dec,97.0,269.0,6.0,25999.0,1,1,1,1,0
61145,Kingdom of Dinza,[Starstrike Studios],Starstrike Studios,Positive,"[Adventure, RPG, Top-Down, 2D, Open World, Pro...",Unknown,2021.0,Dec,90.0,11.0,1.0,59999.0,0,1,0,0,0
59204,Hunahpu Quest. Mechanoid,[Rumata Lab],ValkyrieInitiative,Mostly Positive,"[Indie, Adventure, Platformer, Aliens, Puzzle ...",<128 MB,2018.0,Jan,79.0,24.0,2.0,32999.0,0,1,0,0,0
20710,The Coma: Recut - Deluxe Edition,[Unknown],NaN,<10 reviews,[No Tags],Unknown,NaN,None,NaN,NaN,NaN,162748.0,0,1,1,1,0
67340,Dying Light - Snow Ops Bundle,[Techland],NaN,Very Positive,"[RPG, Action]",4 GB,2021.0,Dec,80.0,95.0,17.0,17499.0,1,1,1,1,0


In [77]:
to_1D(df_dev['developer']).value_counts().head(100)

Unknown                    5214
Ubisoft - San Francisco    1166
 LTD.                      1137
 LLC                       1108
KOEI TECMO GAMES CO.        974
                           ... 
TK.Projects                  50
Techland                     50
Orange_Juice                 49
AMAX Interactive             49
Harmonix Music Systems       49
Length: 100, dtype: int64

#### reformat and remap `publisher`

In [78]:
df_pub = df_dev.copy()
df_pub.loc[df_pub['publisher'].isna()==True,'publisher'] = 'Unknown'
df_pub['publisher'] = df_pub['publisher'].str.split(',')
df_pub.sample(10)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
24679,Shoot 'Em Up Kit,[Tall Studios],[Tall Studios],Mixed,"[Utilities, Early Access, Shoot 'Em Up, Animat...",4 GB,2015.0,Jun,59.0,27.0,1.0,269999.0,0,1,0,0,0
56076,RPG Maker MZ - DS+ Resource Pack,"[Gotcha Gotcha Games, KADOKAWA]",[Degica],<10 reviews,"[RPG, Web Publishing, Design & Illustration]",Unknown,2020.0,Sep,NaN,NaN,2.0,139999.0,0,1,1,0,0
44981,Knights and Castles(1.0),[Pixelabor],[Unknown],<10 reviews,"[Free to Play, RPG]",<128 MB,2021.0,Oct,NaN,NaN,1.0,39999.0,0,1,0,0,0
33519,MadOut Open City,[MadOut Games],[MadOut Games],Mostly Negative,"[Racing, Action, Open World, Simulation, Indie...",<128 MB,2016.0,Mar,37.0,236.0,2.0,69999.0,0,1,1,1,1
16199,Aerofly FS 2 - France VFR - Paris-Ile-de-France,[France VFR],[IPACS],Mostly Positive,"[Indie, Simulation]",8 GB,2020.0,Mar,73.0,15.0,3.0,159999.0,0,1,1,0,0
38415,Agony Soundtrack,[Madmind Studio],[Madmind Studio],<10 reviews,"[Action, Adventure, Indie]",2 GB,2019.0,Feb,NaN,NaN,1.0,32999.0,1,1,0,0,0
44163,VR Time Machine Travelling in history: Visit a...,[William at Oxford],[William at Oxford],<10 reviews,"[Casual, Exploration, VR, Medieval, Singleplayer]",8 GB,2020.0,Aug,NaN,NaN,11.0,69999.0,0,1,0,0,0
52927,Receiver 2 Soundtrack,[Wolfire Games],[Wolfire Games],<10 reviews,[No Tags],Unknown,2020.0,Apr,NaN,NaN,NaN,69999.0,0,0,0,0,0
14529,Super Clown Adventures,"[Tero Lunkka, Valkeala Software]",[Tero Lunkka],Positive,"[Adventure, Arcade, Platformer, Walking Simula...",2 GB,2021.0,Feb,82.0,28.0,1.0,8499.0,0,1,0,0,0
35044,"150,000 B.C.",[carrot],[Garage Games],Mostly Negative,"[Indie, Casual, Adventure, Platformer]",<128 MB,2018.0,May,30.0,33.0,1.0,8499.0,0,1,0,0,0


In [79]:
to_1D(df_pub['publisher']).value_counts().head(60)

Unknown                       10180
Degica                         1078
 LTD.                           906
 Inc.                           862
KOEI TECMO GAMES CO.            788
 Ltd.                           678
Dovetail Games - Trains         514
CAPCOM Co.                      466
Paradox Interactive             425
Big Fish Games                  380
BANDAI NAMCO Entertainment      380
SEGA                            379
Square Enix                     375
Electronic Arts                 325
D3 PUBLISHER                    324
Ubisoft                         319
 LLC                            308
XSEED Games                     289
Marvelous USA                   283
THQ Nordic                      281
Feral Interactive (Mac)         269
2K                              228
Dovetail Games - Flight         228
Slitherine Ltd.                 224
Idea Factory International      212
Feral Interactive (Linux)       204
8floor                          197
NIS America                 

## data understanding

In [80]:
df_final = df_pub.copy()
df_final.sample(5)

,title,developer,publisher,overall_reviews,tags,ram,year,month,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
9383,Upgrade to Movavi Video Suite 2022,[Unknown],[Unknown],<10 reviews,[No Tags],Unknown,NaN,None,NaN,NaN,NaN,978600.0,0,1,0,0,0
28693,Summer Nightmare,[Team Summer Nightmare],[Elkibe Games],Mixed,"[Free to Play, Anime, Visual Novel, Horror, Go...",1 GB,2017.0,Dec,52.0,466.0,2.0,0.0,0,1,1,1,0
37683,Moonlighter: Between Dimensions Original Sound...,[Digital Sun],[Unknown],<10 reviews,"[Action, Adventure, Indie, RPG, Roguelite, Sou...",Unknown,2019.0,Nov,NaN,NaN,NaN,17499.0,0,0,0,0,0
31107,Forgotten Places Collection,[Unknown],[Unknown],<10 reviews,[No Tags],Unknown,NaN,None,NaN,NaN,NaN,51998.0,0,1,1,0,0
16716,Dragon Castle: The Board Game,[Studio Clangore],[Horrible Guild],Positive,"[Board Game, Casual, Tabletop, Asynchronous Mu...",1 GB,2019.0,Dec,92.0,26.0,5.0,64999.0,0,1,1,0,0


In [81]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64887 entries, 0 to 68467
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               64887 non-null  object 
 1   developer           64887 non-null  object 
 2   publisher           64887 non-null  object 
 3   overall_reviews     64887 non-null  object 
 4   tags                64887 non-null  object 
 5   ram                 64887 non-null  object 
 6   year                59907 non-null  float64
 7   month               59903 non-null  object 
 8   user_rating         38188 non-null  float64
 9   total_review        38188 non-null  float64
 10  supported_language  57750 non-null  float64
 11  price               64366 non-null  float64
 12  pegi_rated          64887 non-null  int32  
 13  windows             64887 non-null  int32  
 14  mac                 64887 non-null  int32  
 15  linux               64887 non-null  int32  
 16  VR  

In [82]:
df_final.describe()

,year,user_rating,total_review,supported_language,price,pegi_rated,windows,mac,linux,VR
count,59907.000000,38188.000000,38188.000000,57750.000000,6.436600e+04,64887.000000,64887.000000,64887.000000,64887.000000,64887.000000
mean,2018.223396,76.416963,218.957971,4.774130,8.363383e+04,0.201181,0.965232,0.283632,0.167707,0.009493
std,2.773410,18.492328,4440.137845,5.349063,1.588174e+05,0.400886,0.183193,0.450764,0.373609,0.096971
min,1997.000000,0.000000,10.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.000000,66.000000,18.000000,1.000000,2.499900e+04,0.000000,1.000000,0.000000,0.000000,0.000000
50%,2019.000000,80.000000,40.000000,2.000000,5.299900e+04,0.000000,1.000000,0.000000,0.000000,0.000000
75%,2020.000000,91.000000,122.000000,7.000000,9.599900e+04,0.000000,1.000000,1.000000,0.000000,0.000000
max,2077.000000,100.000000,558563.000000,29.000000,8.990155e+06,1.000000,1.000000,1.000000,1.000000,1.000000


In [83]:
df_final[['overall_reviews','ram']].describe()

,overall_reviews,ram
count,64887,64887
unique,10,14
top,<10 reviews,4 GB
freq,26699,13536


## export csv

In [84]:
df_final.to_csv('for_EDA.csv', index=False)